In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, LSTM
from tensorflow.keras.activations import relu
from tensorflow.keras.metrics import binary_crossentropy
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences 

import warnings
warnings.filterwarnings(action = 'ignore')

In [ ]:
df = pd.read_csv('../Data/spam.csv', encoding= 'latin-1')

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df[~df['Unnamed: 2'].isnull()]['Unnamed: 2']

In [ ]:
df[~df['Unnamed: 2'].isnull()]['v2'] 

In [ ]:
df.isnull().sum()

In [ ]:
df['v2'] = np.where((df['Unnamed: 2'].isnull()), df['v2'], (df['v2'] + ' ' + df['Unnamed: 2']))

In [ ]:
df['v2'] = np.where((df['Unnamed: 3'].isnull()), df['v2'], (df['v2'] + ' ' + df['Unnamed: 3']))

In [ ]:
df['v2'] = np.where((df['Unnamed: 4'].isnull()), df['v2'], (df['v2'] + ' ' + df['Unnamed: 4']))

In [ ]:
df.iloc[95, 1]

In [ ]:
df.isnull().sum()

In [ ]:
df.drop(columns=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], inplace= True, axis = 1)

In [ ]:
df.isnull().sum()

In [ ]:
df['v1'].value_counts()

In [ ]:
sns.set()
plt.figure(figsize = (6, 6))
plt.pie(x = (df['v1'].value_counts().values), labels= (df['v1'].value_counts().keys()), autopct= '%1.1f%%')
plt.show()

In [ ]:
X = df['v2']
y = df['v1']

In [ ]:
X

In [ ]:
y

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
X = X.values.flatten()

In [ ]:
X[1]

In [ ]:
print(list( stopwords.words('english')))

In [ ]:
ps = PorterStemmer()

corpus = []

for i in range(0, len(X)):
    text = re.sub('f[a-zA-Z]', ' ', X[i])
    text = text.lower()
    text = text.split()
    
    text = [ps.stem(j) for j in text if not j in stopwords.words('english')]
    text = ' '.join(text)
    corpus.append(text)
    print(f"Status : {i}")

In [ ]:
corpus

In [ ]:
voc_size = 5000
one_hot_r = [one_hot(i, n = voc_size) for i in corpus]
one_hot_r

In [ ]:
max_len = 0
leng = []

for i in one_hot_r:
    leng.append(len(i))

max_len = max(leng)
max_len

In [ ]:
out_size = len(y.unique())
out_size

In [ ]:
X = np.array(one_hot_r)
y = y

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size= 0.8, random_state= 6, shuffle= True)

In [ ]:
X_train.dtype

In [ ]:
model = Sequential([
    Embedding(input_dim=voc_size, output_dim=50, input_length= max_len),
    LSTM(units= 128),
    Dense(units= out_size, activation= 'relu')
])

model.summary()

In [ ]:
model.compile(optimizer= 'adam', 
             loss = binary_crossentropy,
             metrics= ['acc'])

In [ ]:
model.fit(X_train, y_train, batch_size= 80, verbose= 1, epochs = 5)